In [5]:
import rasterio
import numpy as np
import xarray as xr

In [6]:
image_file = "data/20200814_105841_1053_3B_AnalyticMS.tif"
#20200814_105841_1053_3B_AnalyticMS
# Load red and NIR bands - note all PlanetScope 4-band images have band order BGRN
with rasterio.open(image_file) as src:
    band_red = src.read(3)

with rasterio.open(image_file) as src:
    band_nir = src.read(4)

In [7]:
# Allow division by zero
np.seterr(divide='ignore', invalid='ignore')

# Calculate NDVI. This is the equation at the top of this guide expressed in code
ndvi = (band_nir.astype(float) - band_red.astype(float)) / (band_nir + band_red)

In [11]:
ndvi.shape

numpy.ndarray

In [12]:
import xarray as xr
xr.open_rasterio(image_file) 

ModuleNotFoundError: No module named 'xarray'